In [42]:
!pip install nipype

In [43]:
# import the relevant Python packages
import numpy
import nibabel
import nipype
import matplotlib
import subprocess

In [44]:
!pip install jedi

In [45]:
!pip install pypet2bids

  Using cached pypet2bids-1.0.4-py3-none-any.whl.metadata (1.2 kB)
INFO: pip is looking at multiple versions of pypet2bids to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following versions that require a different python version: 0.0.0 Requires-Python >=3.7,<3.11; 0.0.1 Requires-Python >=3.7,<3.11; 0.0.10 Requires-Python >3.7.1,<3.10; 0.0.11 Requires-Python >3.7.1,<3.10; 0.0.12 Requires-Python >3.7.1,<3.10; 0.0.13 Requires-Python >3.7.1,<3.10; 0.0.14 Requires-Python >3.7.1,<3.10; 0.0.15 Requires-Python >3.7.1,<3.10; 0.0.2 Requires-Python >=3.7,<3.11; 0.0.3 Requires-Python >=3.7,<3.11; 0.0.4 Requires-Python >3.7.1,<3.10; 0.0.4 Requires-Python >=3.7,<3.11; 0.0.5 Requires-Python >=3.7,<3.11; 0.0.6 Requires-Python >3.7.1,<3.10; 0.0.7 Requires-Python >3.7.1,<3.10; 0.0.8 Requires-Python >3.7.1,<3.10; 0.0.9 Requires-Python >3.7.1,<3.10; 1.0.0 Requires-Python >3.7.1,<3.10; 1.0.10 Requires-Python >=3.8,<3.11; 1.0.12 Requires-Python 

## download Phantom ZIP and extract

In [46]:
!wget -O PHANTOMS.zip https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip
!unzip -o PHANTOMS.zip 

--2024-05-29 11:35:15--  https://openneuropet.s3.amazonaws.com/US-sourced-OpenNeuroPET-Phantoms.zip
Resolving openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)... 52.216.32.17, 54.231.133.49, 3.5.16.107, ...
Connecting to openneuropet.s3.amazonaws.com (openneuropet.s3.amazonaws.com)|52.216.32.17|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 115449025 (110M) [application/zip]
Saving to: ‘PHANTOMS.zip’

PHANTOMS.zip        100%[===================>] 110.10M  67.6MB/s    in 1.6s    

2024-05-29 11:35:17 (67.6 MB/s) - ‘PHANTOMS.zip’ saved [115449025/115449025]

Archive:  PHANTOMS.zip
  inflating: __MACOSX/._OpenNeuroPET-Phantoms  
  inflating: OpenNeuroPET-Phantoms/.DS_Store  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._.DS_Store  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._sourcedata  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._sub-SiemensHRRTNRU  
  inflating: __MACOSX/OpenNeuroPET-Phantoms/._code  
  inflating: __MACOSX/OpenNeuroPET-Phanto

In [47]:
import subprocess
check_for_tree = subprocess.run(['tree', '--version'], capture_output=True)
if check_for_tree.returncode == 0:
    pass
else:
    import platform
    operating_system = platform.system()
    if operating_system == 'Linux':
        subprocess.run("apt-get install tree -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install tree", shell=True)
    else:
        print("You're on your own windows user.")

In [48]:
!tree OpenNeuroPET-Phantoms/ --filelimit 15

OpenNeuroPET-Phantoms/
├── README
├── code
│   ├── matlab_conversions.m
│   └── python_conversions.sh
├── dataset_description.json
├── sourcedata
│   ├── GeneralElectricAdvance-NIMH
│   │   ├── 2d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d375_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   ├── 3d_unif_lt_ramp  [35 entries exceeds filelimit, not opening dir]
│   │   └── long_trans  [35 entries exceeds filelimit, not opening dir]
│   ├── GeneralElectricSignaPETMR-NIMH  [89 entries exceeds filelimit, not opening dir]
│   ├── SiemensBiographPETMR-NIMH
│   │   ├── AC_TOF  [150 entries exceeds filelimit, not opening dir]
│   │   ├── CT  [148 entries exceeds filelimit, not opening dir]
│   │   └── NAC  [148 entries exceeds filelimit, not opening dir]
│   └── SiemensHRRT-JHU
│       └── Hoffman.v
├── sub-GeneralElectricAdvanceLongNIMH
│   └── pet
│       ├── sub-GeneralElectricAdvanceLongNIMH_pet.json
│       └── sub-GeneralElectricAdvanc

In [49]:
# check for dcm2niix
check_dcm2niix = subprocess.run("dcm2niix -h", shell=True, capture_output=True)
if check_dcm2niix.returncode == 0:
    print('dcm2niix is installed')
    # set dcm2niix path as this is running in an ipython notebook
    dcm2niix_path = subprocess.run("which dcm2niix", shell=True, capture_output=True)
    subprocess.run(f"dcm2niix4pet --set-dcm2niix-path {dcm2niix_path.stdout.decode()}", shell=True)
else:
    print('dcm2niix is not installed')
    import platform
    operating_system = platform.system()

    if operating_system == 'Linux':
        subprocess.run("apt-get install dcm2niix -y", shell=True)
    elif operating_system == 'Darwin':
        subprocess.run("brew install dcm2niix", shell=True)
    else:
        print("You're on your own windows user.")

dcm2niix is installed


In [50]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder

2024-05-29 11:35:20,503 - pypet2bids - WARNING - dcm2niix found on system path, but dcm2niix path is also set in ~/.pet2bidsconfig. Defaulting to dcm2niix path set in config at /usr/local/bin/dcm2niix (dcm2niix4pet.py:364)
2024-05-29 11:35:20,507 - pypet2bids - WARNING - Minimum version v1.0.20220720 of dcm2niix is recommended, found installed version v1.0.20211006 at /usr/local/bin/dcm2niix. (dcm2niix4pet.py:217)


In [51]:
!tree ./mynewfolder

./mynewfolder
├── PET_Brain_AC_TOF_resbrain_20210504071146_3.json
└── PET_Brain_AC_TOF_resbrain_20210504071146_3.nii.gz

0 directories, 2 files


In [52]:
!dcm2niix4pet ./OpenNeuroPET-Phantoms/sourcedata/SiemensBiographPETMR-NIMH/AC_TOF -d mynewfolder2 --kwargs TimeZero=ScanStart Manufacturer=Siemens ManufacturersModelName=Biograph InstitutionName="NIH Clinical Center" BodyPart=Phantom Units=Bq/mL TracerName=none TracerRadionuclide=F18 InjectedRadioactivity=81.24 SpecificRadioactivity=13019.23 ModeOfAdministration=infusion FrameTimesStart=0 AcquisitionMode="list mode" ImageDecayCorrected=true ImageDecayCorrectionTime=0 AttenuationCorrection=MR-corrected FrameDuration=300 FrameTimesStart=0

2024-05-29 11:35:21,477 - pypet2bids - WARNING - dcm2niix found on system path, but dcm2niix path is also set in ~/.pet2bidsconfig. Defaulting to dcm2niix path set in config at /usr/local/bin/dcm2niix (dcm2niix4pet.py:364)
2024-05-29 11:35:21,481 - pypet2bids - WARNING - Minimum version v1.0.20220720 of dcm2niix is recommended, found installed version v1.0.20211006 at /usr/local/bin/dcm2niix. (dcm2niix4pet.py:217)


## validate BIDS dataset
https://www.npmjs.com/package/bids-validator

In [53]:
!pip install bids_validator

In [54]:
from bids_validator import BIDSValidator

In [55]:
BIDSValidator().is_bids('.') # not up to date, use browser version for now

False

## run only if freesurfer is installed

In [56]:
!mkdir PETprocessing

mkdir: PETprocessing: File exists


In [57]:
!cd PETprocessing

In [72]:
!if [ -d "PET_pipelines" ]; then pushd PET_pipelines && git pull; else git clone https://github.com/openneuropet/PET_pipelines.git; fi

Cloning into 'PET_pipelines'...
remote: Enumerating objects: 69, done.
remote: Counting objects: 100% (69/69), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 69 (delta 20), reused 35 (delta 5), pack-reused 0
Receiving objects: 100% (69/69), 33.16 KiB | 2.37 MiB/s, done.
Resolving deltas: 100% (20/20), done.


In [73]:
!cd PET_pipelines/pyPetSurfer 

In [85]:

#!wget https://www.dropbox.com/sh/69dwtnv29wd7jlx/AADnw5FvAANpvzKAxVQTnyhBa?dl=0
!chmod +x ds001421-1.4.1.sh
!mkdir ds001421-download/
!cp ds001421-1.4.1.sh ds001421-download/ && cd ds001421-download/ && ./ds001421-1.4.1.sh
!rm ds001421-1.4.1.sh && cd ..

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   719  100   719    0     0   3476      0 --:--:-- --:--:-- --:--:--  3490
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   229  100   229    0     0   3128      0 --:--:-- --:--:-- --:--:--  3136
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   958  100   958    0     0  15076      0 --:--:-- --:--:-- --:--:-- 15206
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   364  100   364    0     0   3196      0 --:--:-- --:--:-- --:--:--  3221
  % Total    % Received % Xferd  Average Speed   Tim

In [87]:
!ls PET_pipelines/pyPetSurfer

LICENSE       README.md     example.py    pet_surfer.py


In [88]:
#!pip install mri_coreg

ERROR: Could not find a version that satisfies the requirement mri_coreg (from versions: none)
ERROR: No matching distribution found for mri_coreg


In [97]:
from PET_pipelines.pyPetSurfer import pet_surfer

#from PET_pipelines.pyPetSurfer import example


ModuleNotFoundError: No module named 'pet_surfer'

In [ ]:
!ls .
